#### Load data from a single session

In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from ephys import *
from ephys_utils import select_spikes_by_trial, transform_spike_data, find_template_for_clusters
from spatial_analysis import *

pos_sample_rate = 50
path_to_session = '/home/isabella/Documents/isabella/jake/recording_data/r1364/2023-06-14'

# Load spike times and position for a single session
obj = ephys(recording_type = 'nexus', path = path_to_session)

obj.load_spikes('good')
obj.load_pos([i for i, s in enumerate(obj.trial_list)])

Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1364/2023-06-14/230614_r1364_raw_open-field_1.pos
893 LED swaps detected and fixed


/data/isabella/jake/ephys_analysis/pyScan/postprocessing/../postprocessing/postprocess_pos_data.py:219: RuntimeWarning: invalid value encountered in remainder
  dir_disp = np.mod(np.arctan2(-pos.iloc[1, 1:].values + pos.iloc[1, :-1].values, pos.iloc[0, 1:].values - pos.iloc[0, :-1].values) * 180 / np.pi, 360)


Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1364/2023-06-14/230614_r1364_raw_t-maze_1.pos
147 LED swaps detected and fixed
Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1364/2023-06-14/230614_r1364_raw_open-field_2.pos
631 LED swaps detected and fixed
Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1364/2023-06-14/230614_r1364_raw_t-maze_2.pos
95 LED swaps detected and fixed


#### Filter good cells to only include CA1 pyramidal cells following criteria for inclusion in Wills et al., 2010

In [2]:
cluster_info = obj.spike_data['cluster_info'] #Get cluster info from phy

## Get cluster depths and exclude any outside of 0 +-200um
cluster_info = cluster_info[cluster_info['depth'].between(-200, 200)]

## Filter for mean firing rate - CHECK EXACT VALUES
cluster_info = cluster_info[cluster_info['fr'].between(1, 10)]

## Filter for spike width from template
# Find kilosort template for each cluster (closely approximates spike width) and add to dataframe
template_per_cluster = find_template_for_clusters(obj.spike_data['spike_clusters'], obj.spike_data['spike_templates'])
template_df = pd.DataFrame.from_dict(template_per_cluster, orient = 'index')
template_df.columns = ['template_id']

cluster_info = cluster_info.join(template_df, how = 'left')

# Load templates.npy
templates = np.load(f'{path_to_session}/{path_to_session[-8:-6]}{path_to_session[-5:-3]}{path_to_session[-2:]}_sorting_ks2_custom/templates.npy')
# Load inverse whitening matrix and apply to unwhiten templates
whitening_matrix_inv =  np.load(f'{path_to_session}/{path_to_session[-8:-6]}{path_to_session[-5:-3]}{path_to_session[-2:]}_sorting_ks2_custom/whitening_mat_inv.npy')
unwhitened_templates = np.einsum('ijk,kl->ijl', templates, whitening_matrix_inv)

# Add template values to dataframe
cluster_info['template'] = cluster_info.apply(lambda row: templates[int(row['template_id']), :, int(row['ch'])], axis=1)

# Work out template width peak to trough
sampling_rate = obj.spike_data['sampling_rate']

spike_width_samples = cluster_info['template'].apply(
    lambda x: np.abs(np.argmax(x) - np.argmin(x))
)

# Convert spike width to microseconds and add to dataframe
spike_width_microseconds = (spike_width_samples/sampling_rate)*1000000
cluster_info['spike_width_microseconds'] = spike_width_microseconds

# Filter for spike width > 300us as in Wills et al., 2010
cluster_info = cluster_info[cluster_info['spike_width_microseconds'] > 300]

print(f'{len(cluster_info.index)} cells retained of {len(obj.spike_data["cluster_info"].index)} good cells from phy')

12 cells retained of 20 good cells from phy


In [3]:
# Reload spike data only for included cells
clusters_inc = list(cluster_info.index)
obj.load_spikes(clusters_to_load = clusters_inc)

print(f'Reloaded spike data for {len(clusters_inc)} candidate CA1 pyramidal cells')

Reloaded spike data for 12 candidate CA1 pyramidal cells


In [4]:
# Generate autocorrelograms and burst index for each cluster
from burst_index_and_autocorrelograms import *

spike_times_inc = obj.spike_data['spike_times']
spike_clusters_inc = obj.spike_data['spike_clusters']

autocorrelograms, burst_indices = compute_autocorrelograms_and_burst_indices(spike_times_inc, 
                                                                             spike_clusters_inc, 
                                                                             bin_size = 0.001, #1ms
                                                                             time_window = 0.05, #50ms
                                                                             burst_threshold = 0.01 #10ms
                                                                            )
cluster_info['burst_index'] = burst_indices.values()

In [5]:
# Plot the autocorrelograms
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

plot_autocorrelograms_with_dropdown(autocorrelograms)

interactive(children=(Dropdown(description='Cluster ID:', options=(304, 307, 311, 319, 320, 323, 328, 339, 344…

In [6]:
# Loop through trials and generate rate maps
rate_maps = {}
occupancy = {}

for trial, trial_name in enumerate(obj.trial_list):
    
    # Select spikes for current trial and transform to create a dict of {cluster: spike_times, cluster:spike_times}
    current_trial_spikes = select_spikes_by_trial(obj.spike_data, trial, obj.trial_offsets)
    current_trial_spikes = transform_spike_data(current_trial_spikes)
    
    
    rate_maps[trial], occupancy[trial] = generate_rate_maps(spike_data = current_trial_spikes,
                               positions = obj.pos_data[trial]['xy_position'],  
                               ppm = 400, 
                               x_bins = 50,
                               y_bins = 50,
                               dt = 1.0,
                               smoothing_window = 10)


In [7]:
#Plot rate maps
interactive_cluster_plot(rate_maps, title_prefix="Rate Maps")


interactive(children=(Dropdown(description='Cluster ID:', options=(304, 307, 311, 319, 320, 323, 328, 339, 344…

In [8]:
from scipy.stats import entropy

def calculate_spatial_information(rate_maps, occupancy, dt=1.0):
    """
    Calculate Skaggs' spatial information score for given rate maps and occupancy.
    
    Parameters:
    - rate_maps: dict
        Dictionary containing smoothed rate maps organized by clusters.
    - occupancy: np.ndarray
        2D array indicating occupancy of each bin.
    - dt: float
        Time window for spike count.
        
    Returns:
    - skaggs_info_dict: dict
        Dictionary containing Skaggs' spatial information scores organized by clusters.
    """
    
    skaggs_info_dict = {}
    
    # Calculate the total time spent in the environment
    total_time = np.nansum(occupancy) * dt

    for cluster, rate_map in rate_maps.items():
        
        # Calculate mean firing rate across all bins
        mean_firing_rate = np.nansum(rate_map * occupancy) / total_time

        # Calculate probability of occupancy for each bin
        prob_occupancy = occupancy / np.nansum(occupancy)

        # Calculate Skaggs' spatial information score
        non_zero_idx = (rate_map > 0) & (prob_occupancy > 0)
        skaggs_info = np.nansum(
            prob_occupancy[non_zero_idx] *
            rate_map[non_zero_idx] *
            np.log2(rate_map[non_zero_idx] / mean_firing_rate)
        )

        skaggs_info_dict[cluster] = skaggs_info
            
    return skaggs_info_dict

# Calculate spatial information - NEEDS ADJUSTING TO HANDLE NAN VALUES
spatial_info = {}

for i in rate_maps.keys():
    spatial_info[i] = calculate_spatial_information(rate_maps[i], occupancy[i], dt = 1)
spatial_info

{0: {304: 0.00870528442709359,
  307: 0.021597820683300805,
  311: 0.007845765701137363,
  319: 0.034017693243491465,
  320: 0.006918015696174691,
  323: 0.049586851775867316,
  328: 0.01812516713329497,
  339: 0.04041423136094044,
  344: 0.024992576180382794,
  356: 0.008480155839269792,
  367: 0.020442169899537933,
  369: 0.005964909392598753},
 1: {304: 0.013696505957652611,
  307: 0.021408739701547566,
  311: 0.006933053602450876,
  319: 0.009152825222202246,
  320: 0.012894410287277603,
  323: 0.016935527829033428,
  328: 0.009337559652727093,
  339: 0.016358279892725307,
  344: 0.02466583219073954,
  356: 0.00386571183776087,
  367: 0.012256209722344997,
  369: 0.007690658724720244},
 2: {304: 0.010752579481752457,
  307: 0.015407363398510662,
  311: 0.006309856947500501,
  319: 15890746.02701045,
  320: 15890745.848372359,
  323: 15890745.816983908,
  328: 0.019526566314711065,
  339: 0.017606442563150623,
  344: 0.04033978272208272,
  356: 0.0038760907075269463,
  367: 0.028903

In [9]:
len(rate_maps)

4